# Face Recognition Assignment #1

## Problem Statement 

We intend to perform face recognition. Face recognition means that for a given image
you can tell the subject id. Our database of subject is very simple. It has 40 subjects.
Below we will show the needed steps to achieve the goal of the assignment.

### Step 1: Download the dataset and understand the format 

In [1]:
from os import listdir
from PIL import Image as PImage

def loadImages(path):
    # return array of images
    foldersList = listdir(path)
    loadedImages = []
    for folder in foldersList :
        imagesList = listdir(path+folder)
        for image in imagesList:
            img = PImage.open(path +folder+'/'+ image)
            loadedImages.append(img)
    return loadedImages

In [2]:
path = "./orl_faces/"
# your images in an array
imgs = loadImages(path)

### Step 2: Generate the Data Matrix and the Label vector 

In [3]:
import numpy as np
dataMatrix = np.arange(4121600).reshape(400,10304)
j=0
label = []
for i in range(0,400) :
    if(i%10 == 0):
        j = j+1
    dataMatrix[i] = np.array(imgs[i]).flatten()
    label.append(j)
label

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,


### Step 3: Splitting the dataset into training and test sets

In [4]:
trainSet=np.arange(200*10304).reshape(200,10304)
testSet=np.arange(200*10304).reshape(200,10304)
trainLabel=[]
testLabel=[]
j,k=0,0
for i in range(0,400):
    if(i%2==0):
        testSet[j]=dataMatrix[i]
        testLabel.append(label[i])
        j+=1
    else:
        trainSet[k]=dataMatrix[i]
        trainLabel.append(label[i])
        k+=1


### Step 4: Classification using PCA

In [5]:
dataMean=np.mean(trainSet,axis=0)
centerDataMatrix=trainSet-dataMean
covMatrix=(1/200)*(centerDataMatrix @ centerDataMatrix.T)
eigVal,eigVectMatrix=np.linalg.eigh(covMatrix)
eigVal=np.flip(eigVal,axis=0)
eigVectMatrix=np.flip(eigVectMatrix,axis=0)


In [6]:
def computeDimensionality(eigVal,threshold):
    totalDataVariance=np.sum(eigVal)
    varianceFraction,numDim,eigValSum=0,1,0
    
    while(varianceFraction<threshold):
        eigValSum+=eigVal[numDim-1]
        varianceFraction=eigValSum/totalDataVariance
        numDim+=1
    return numDim

In [7]:
dimAlpha1=computeDimensionality(eigVal,0.8)
dimAlpha2=computeDimensionality(eigVal,0.85)
dimAlpha3=computeDimensionality(eigVal,0.9)
dimAlpha4=computeDimensionality(eigVal,0.95)


In [8]:
projMat1=eigVectMatrix[:,0:dimAlpha1]
projMat2=eigVectMatrix[:,0:dimAlpha2]
projMat3=eigVectMatrix[:,0:dimAlpha3]
projMat4=eigVectMatrix[:,0:dimAlpha4]


In [9]:
reducedDimTrain1=projMat1.T @ trainSet
reducedDimTrain2=projMat2.T @ trainSet
reducedDimTrain3=projMat3.T @ trainSet
reducedDimTrain4=projMat4.T @ trainSet


In [10]:
reducedDimTest1=projMat1.T @ testSet
reducedDimTest2=projMat2.T @ testSet
reducedDimTest3=projMat3.T @ testSet
reducedDimTest4=projMat4.T @ testSet


### Step 5: Classification using LDA

In [11]:
dataClassList=[]
for i in range(0,401):
    if(i%10==0 and i!=0):
        dataClassList.append(dataMatrix[i-10:i,:])

In [39]:
mean=np.mean(dataClassList,axis=1)
mean

array([[ 47.8,  49.7,  47.2, ...,  49.9,  46.9,  49. ],
       [ 35.7,  35.8,  35.8, ...,  84.6,  85.7,  84.4],
       [108.3, 109.4, 110.5, ...,  84.7,  80.4,  83.7],
       ...,
       [121. , 122.4, 121. , ...,  91.4,  96.5,  95.6],
       [ 42.5,  43.7,  41.7, ...,  25.6,  23. ,  22. ],
       [ 50.5,  51. ,  52.8, ...,  42.8,  44.9,  45.7]])

In [26]:
withinClassScatterMatrix = np.zeros((10304,10304))
for i in range(0,40):
    diffMatrix = (dataClassList[i]-mean[i])
    withinClassScatterMatrix += diffMatrix.T @ diffMatrix

(10304, 10304)

In [38]:
overallSampleMean = 1/40* np.sum(mean, axis =0)
overallSampleMean

array([85.6175, 85.5775, 85.925 , ..., 76.87  , 75.865 , 75.2275])

In [42]:
betweenClassScatterMatrix = np.zeros((10304,10304))
for i in range(0,40):
    diffVector = (mean[i]- overallSampleMean)
    betweenClassScatterMatrix  += np.outer(diffVector,diffVector) 

In [44]:
np.linalg.eig(np.linalg.inv(withinClassScatterMatrix)@ betweenClassScatterMatrix)

(array([ 9.84405173e+18+0.j       ,  2.62869630e+18+0.j       ,
        -1.13348817e+18+0.j       , ...,  5.69299441e+00+0.j       ,
         1.53618240e+01+4.9234791j,  1.53618240e+01-4.9234791j]),
 array([[-0.01527507+0.j        , -0.07539944+0.j        ,
          0.14100898+0.j        , ...,  0.00360501+0.j        ,
          0.00137671+0.00314661j,  0.00137671-0.00314661j],
        [-0.03872975+0.j        , -0.13755672+0.j        ,
          0.07987574+0.j        , ..., -0.0016153 +0.j        ,
         -0.00473542-0.00062325j, -0.00473542+0.00062325j],
        [-0.04793161+0.j        ,  0.05699947+0.j        ,
         -0.03832383+0.j        , ..., -0.00915387+0.j        ,
          0.00642665+0.00187265j,  0.00642665-0.00187265j],
        ...,
        [-0.02589195+0.j        ,  0.02579482+0.j        ,
         -0.00969517+0.j        , ...,  0.00365483+0.j        ,
         -0.000683  +0.00115062j, -0.000683  -0.00115062j],
        [-0.00065264+0.j        , -0.00974578+0.j       